In [1]:
%load_ext autoreload
%autoreload 2

In [49]:
from AutoLLM.prompt_generators.classifier_prompt_generator import ClassifierPromptGenerator

config = {
    "sample_subprompt": [
        {
            "name": "Context",
            "description": "Context",
        },
        {
            "name": "Reasoning",
            "description": "Reasoning for the answer",
        },
        {
            "name": "Answer",
            "description": "Answer as positive or negative only",
        }
    ],
    "chat": False,
    "separator": "\n\n=====\n\n",
}
few_shot_examples = [
    {
        "Context": "I am happy",
        "Reasoning": "The label is positive because of the word happy.",
        "Answer": "positive"
    },
    {
        "Context": "I am angry",
        "Reasoning": "The label is negative because of the word angry.",
        "Answer": "negative"
    },
    {
        "Context": "I am sad",
        "Reasoning": "The label is negative because of the word sad.",
        "Answer": "negative"
    },
]

input = {
    "Context": "I am satisfied with the product",
}

cpg = ClassifierPromptGenerator()
cpg.load_config(config)
cpg.load_library()

cpg.build_component_system_message('I am a classifier that classifies the sentiment of a given text as positive or negative.')
cpg.build_component_instruction(
    instruction='Classify the sentiment of the following text as positive or negative.',
    chain_of_thought='Provide your reasoning for the answer.',
    echo='Repeat the question before you answer.',
)
cpg.build_component_format()
# cpg.build_component_few_shot_examples(few_shot_examples)
cpg.build_component_input_text(input)
cpg.build_component_guide()
print(cpg.build_prompt())

# cpg.sample_subprompt.build_multiple_examples(few_shot_examples)

I am a classifier that classifies the sentiment of a given text as positive or negative.

=====

Classify the sentiment of the following text as positive or negative. Provide your reasoning for the answer. Repeat the question before you answer.

=====

Format:

Context: Context

Reasoning: Reasoning for the answer

Answer: Answer as positive or negative only

=====

Context: I am satisfied with the product

Reasoning: 


In [ ]:
# config = {
    "instruction": ""
}